# Querying in a corpus of documents (of the web, for that matter?)

### The purpose of this class is to understand how a corpus of documents can be indexed in a 'simple' way. After that, we can retrieve the most relevant documents for a given query, or we could also try to perform some clustering.

### To this end, we are going to use a set of sonnets by Shakespeare

In [1]:
# path to the file. You should point this to your data
file = '../data/shakespeare.txt'

# see the document, using the 'head' command
!head -n60 $file

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lus

### Note that if we simply invoke the method `sc.textFile` on the input file, we will read the data line by line. For this excercise we want to create a corpus of documents; hence, we will read the data paragraph by paragraph, so that each paragraph will be like a document.

### The method `newAPIHadoopFile` is quite advanced, so don't worry if you are not familiar with it :)

In [5]:
# read data by paragraphs
paragraphs = sc.newAPIHadoopFile(file, "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
    "org.apache.hadoop.io.LongWritable", "org.apache.hadoop.io.Text",
    conf={"textinputformat.record.delimiter": '\n\n'}).map(lambda l:l[1])

paragraphs.take(5)

[u'1609',
 u'THE SONNETS',
 u'by William Shakespeare',
 u'',
 u"                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee."]

### One we have our corpus of documents, we have to perform some cleaning on it

In [6]:
# clean the data: we want to remove spaces (larger than one), empty lines and punctuation
import re

cleanParagraphs = (paragraphs
# remove punctuation 
.map(lambda p: re.sub('[^a-zA-Z0-9 ]','',p.lower().strip())) 
# replace multiple spaces by single space
.map(lambda p: re.sub('[ ]+',' ',p))
# remove empty lines from the RDD
.filter(lambda p: p!='') )

In [7]:
# visualize some of the 'documents' to double check that everything went well
cleanParagraphs.take(10)

[u'1609',
 u'the sonnets',
 u'by william shakespeare',
 u'1 from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee',
 u'2 when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a tattered weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how muc

### For the purpose of querying in the corpus, it's convinient to index the documents with `zipWithIndex`. See the differece with `zipWithUniqueId`.

### Also, we are going to persist the data in main memory, since we may want to perform several queries. The  level of persistance can be changed usgin the `StorageLevel`, for instance `persist(StorageLevel.MEMORY_AND_DISK)`. In order to see the level of persistance you can use the method `getStorageLevel()`.

In [8]:
# add index to 'documents'. 
cleanParagraphsWithIndex = (cleanParagraphs
                            .zipWithIndex() 
                            .map(lambda (doc,i): (i,doc)))
# persist the data in RAM 
cleanParagraphsWithIndex.cache()

PythonRDD[12] at RDD at PythonRDD.scala:43

In [9]:
cleanParagraphsWithIndex.take(3)

[(0, u'1609'), (1, u'the sonnets'), (2, u'by william shakespeare')]

In [10]:
# number of documents (that is, paragraphs) in our corpus:
print 'There are %d "documents"' % cleanParagraphsWithIndex.count()

There are 6376 "documents"


### Our next task is to *label* each document. One simple approach is to use the well-known  *term frequency–inverse document frequency*, or TF-IDF. Recall what we have seen in the class, and have a look at wikipedia!!!

### As we have seen during the lesson, PySpark implementation of tf-idf has some problems. Hence, I leave here a custom implementation. This works well for the data we are using, but it's not scalable!!! (it will fail on a larger corpus of documents). At the end of the excerssise I will suggest some improvements ;)

### In the following, we will be calculating separatedly the TF and IDF terms in a scalable way. Then, we will combine them by *collecting* the IDF RDD (i.e., bringing the data to memory). As I've said, this solution works here, but might fail in a larger corpus...

## 1. TF: term frequency in each document

### Basically, we want a histogram of words for each document in the corpus. In order to leverage the importance of larger documents, we use log-scale instead of the raw frequency of the words.

In [11]:
# calculate document frequency:
from math import log
def create_hist_for_document(doc):
    '''
    This method accepts a string (doc), and returns its histogram of words 
    '''
    # split input string into array of words
    listOfWords = doc.split(' ')
    # return a dictionary of key = word and value = frequency of that word
    dicc = dict((x, listOfWords.count(x)) for x in listOfWords)
    # convert to logarithmic scale: this step is optional, we can use directly the frequencies
    dicc = {k:(1+log(dicc[k]) if dicc[k]!=0 else 0.0) for k in dicc}
    return dicc

# we use assert to check the solution
assert create_hist_for_document('1 1 1 23 23 hola hola hola 1 a') == \
    {'1': 1+log(4), 'a': 1, '23': 1+log(2), 'hola': 1+log(3)}

In [12]:
# apply the function create_hist_for_document to each 'document' in the RDD
tf = cleanParagraphsWithIndex.map(lambda (i,doc): (i,create_hist_for_document(doc)))

In [13]:
# check everything went well
tf.take(4)

[(0, {u'1609': 1.0}),
 (1, {u'sonnets': 1.0, u'the': 1.0}),
 (2, {u'by': 1.0, u'shakespeare': 1.0, u'william': 1.0}),
 (3,
  {u'1': 1.0,
   u'a': 1.0,
   u'abundance': 1.0,
   u'and': 2.09861228866811,
   u'art': 1.0,
   u'as': 1.0,
   u'be': 1.0,
   u'bear': 1.0,
   u'beautys': 1.0,
   u'bright': 1.0,
   u'bud': 1.0,
   u'buriest': 1.0,
   u'but': 1.6931471805599454,
   u'by': 1.6931471805599454,
   u'churl': 1.0,
   u'content': 1.0,
   u'contracted': 1.0,
   u'creatures': 1.0,
   u'cruel': 1.0,
   u'decease': 1.0,
   u'desire': 1.0,
   u'die': 1.0,
   u'due': 1.0,
   u'eat': 1.0,
   u'else': 1.0,
   u'eyes': 1.0,
   u'fairest': 1.0,
   u'famine': 1.0,
   u'feedst': 1.0,
   u'flame': 1.0,
   u'foe': 1.0,
   u'fresh': 1.0,
   u'from': 1.0,
   u'fuel': 1.0,
   u'gaudy': 1.0,
   u'glutton': 1.0,
   u'grave': 1.0,
   u'heir': 1.0,
   u'herald': 1.0,
   u'his': 1.6931471805599454,
   u'in': 1.0,
   u'increase': 1.0,
   u'lies': 1.0,
   u'lights': 1.0,
   u'making': 1.0,
   u'makst': 1.0,
 

## 2. Inverse document frequency (IDF):

### After having obtained the frequency of words in each document (the so-called TF), we may want to remove terms that are too common. For instance, words like *a*, *and* or *the* are very unlikely to represent a document. Recall from the classroom that such words are known as [*stop-words*](https://en.wikipedia.org/wiki/Stop_words).

### The IDF index of a word is inversely proportional to the occurrance of that word in the corpus and thus, is useful for removing terms that appear frequently in our corpus. Note that we only take into account whether the word appear in a document, not how many times!!!

In [20]:
from operator import add
from math import log

# get unique words in a document
def word_in_doc(doc):
    '''
    This method return the unique words in a document. Since we want to 
    calculate the occurrance of words among documents, we return a list of 
    tuples (word,1). This structure will be useful later on in the reduce phase.
    '''
    list_words = doc.split(' ')
    unique_words = set(list_words)
    # emit a 1 for each word:
    words_with_ones = map(lambda w: (w,1),unique_words)
    return words_with_ones

# note that we only count 'tal' once (because the sentence passed to words_in_doc represents a single 'document')
assert word_in_doc('hola que tal 123 tal') == [('tal', 1), ('123', 1), ('que', 1), ('hola', 1)]

# calculate inverse document frequency
def inverse_doc_freq(rdd):
    '''
    This method calculates the inverse document frequency for each term in each document
    Input: and rdd consisting of tuples of (doc_index, doc)
    
    '''
    nunmberOfDocs = rdd.count()
    
    # get an RDD of words with the number of documents this word appears in
    freqWordsInDocs = (rdd.flatMap(lambda (i,doc): word_in_doc(doc))
                       .reduceByKey(add))
    # inverse doc frequency: an RDD of tuples of the form (key=word,value=idf)
    idf = freqWordsInDocs.map(lambda (w,f): (w,log(1+nunmberOfDocs/float(1+f))))
    return idf

# Check that everything went well (you can try other sentences, but be sure to change the assert statement accordingly!)
test_corpus = sc.parallelize([(0,'hola que tal 123 tal'),(1,'hola mi amor amor'),(2,'hola que haces amor')])
assert inverse_doc_freq(test_corpus).collectAsMap()  == {'haces': log(1+3.0/(1+1)), 
           'mi': log(1+3.0/(1+1)), '123': log(1+3.0/(1+1)), 'que': log(1+3.0/(1+2)), 
           'tal': log(1+3.0/(1+1)), 'amor': log(1+3.0/(1+2)), 'hola': log(1+3.0/(1+3))}

## 3. TF-IDF

### Now, we have to combine the TF and IDF RDDs. Remember that TF is an RDD of tuples:
###    (index_of_doc, dictionary of word frequencies)
### and IDF is a RDD of tuples: 
###    (word, idf)

### There are several approaches to combine both RDDs. For this excersise, we will follow the simplest approach: collect the IDF RDD as a dictionary. It should be clear that, for a sufficiently large corpus, bringing this data to memory can cause a memoryOutOfBounds exception

In [21]:
# this might be unsafe!!!
idf = inverse_doc_freq(cleanParagraphsWithIndex).collectAsMap()

In [22]:
print 'Some idf: %s' % [(i,idf[i]) for i in idf][:10]

Some idf: [(u'fawn', 6.123432217756322), (u'', 6.969477337610134), (u'nunnery', 7.662154335573845), (u'redeemst', 8.067462667010057), (u'woods', 5.929745576710511), (u'spiders', 6.969477337610134), (u'hanging', 5.155164546504638), (u'offendeth', 8.067462667010057), (u'beadsmen', 8.067462667010057), (u'scold', 6.56448219113042)]


In [23]:
# put toghether the tf and the idf
tfidf = tf.map(lambda (i,dicc): (i,{k:dicc[k]*idf[k] for k in dicc if k in idf}))
tfidf.cache()

PythonRDD[59] at RDD at PythonRDD.scala:43

In [26]:
# Note that until we don't execute this cell, nothing is actually done
tfidf.take(4)
# also, note that if you execute the cell again, the result appears inmediatly, because we persisted the RDD tfidf

[(0, {u'1609': 7.374629015218945}),
 (1, {u'sonnets': 6.969477337610134, u'the': 1.0498658171066235}),
 (2,
  {u'by': 1.6648624464339654,
   u'shakespeare': 5.102832630920091,
   u'william': 4.454502322995931}),
 (3,
  {u'1': 4.7797457706673905,
   u'a': 1.2134235539353357,
   u'abundance': 6.123432217756322,
   u'and': 2.114356047254294,
   u'art': 2.5707599490570026,
   u'as': 1.596515267393861,
   u'be': 1.4896053641677396,
   u'bear': 2.806459785645822,
   u'beautys': 5.468685043514473,
   u'bright': 4.675317945890173,
   u'bud': 6.1973836831408065,
   u'buriest': 8.067462667010057,
   u'but': 2.530633792033937,
   u'by': 2.8188571571998016,
   u'churl': 6.459278280280682,
   u'content': 3.975228671495789,
   u'contracted': 6.2772698595469025,
   u'creatures': 5.053133931046207,
   u'cruel': 4.596051816651253,
   u'decease': 6.815483336200744,
   u'desire': 3.6225870148083623,
   u'die': 3.029119289586728,
   u'due': 4.566056929747677,
   u'eat': 4.122669748270193,
   u'else': 2.95

## 4. Querying

### We are finally in a position to perform our queries on the corpus. The idea is to retrieve the most relevant docuemnts for a given set of words (i.e, the query)

In [29]:
import numpy as np
def get_paragraphs(tfidf =None, query = None, numberOfDocs = 10):
    return (tfidf
           .map(lambda (i,dicc): (i,sum({dicc[k] if k in dicc else -1 for k in query}) ))
           .takeOrdered(numberOfDocs,key=lambda x: -x[1]))

def return_docs(query = None, corpus = None, tfidf = None, numberOfDocs =10):
    p = get_paragraphs(tfidf, query, numberOfDocs)
    print 'Top-%s paragraphs are %s' % (numberOfDocs,p)
    
    indexes = [i[0] for i in p]
    return corpus.filter(lambda (i,doc): i in indexes).collect()

In [31]:
query = ['william','shakespeare']
return_docs(query=query, corpus=cleanParagraphsWithIndex,tfidf=tfidf,numberOfDocs=5)

Top-5 paragraphs are [(3998, 17.22355811195959), (706, 15.517518234595325), (2366, 10.623747242651353), (1812, 9.629753774960644), (2, 9.557334953916023)]


[(2, u'by william shakespeare'),
 (706,
  u'touchstone it is meat and drink to me to see a clown by my troth we that have good wits have much to answer for we shall be flouting we cannot hold william good evn audrey audrey god ye good evn william william and good evn to you sir touchstone good evn gentle friend cover thy head cover thy head nay prithee be coverd how old are you friend william five and twenty sir touchstone a ripe age is thy name william william william sir touchstone a fair name wast born i th forest here william ay sir i thank god touchstone thank god a good answer art rich william faith sir so so touchstone so so is good very good very excellent good and yet it is not it is but so so art thou wise william ay sir i have a pretty wit touchstone why thou sayst well i do now remember a saying the fool doth think he is wise but the wise man knows himself to be a fool the heathen philosopher when he had a desire to eat a grape would open his lips when he put it into his mo

### Check out that the word *william* appears several times in the last documents. Thus, we are actually retrieving relevant documents :) 

### However, the last document has more than 20 occurrancies of the word *william*. It seems reasonable that this document should have been returned in a higher position, right? Can you think of an explanation for this?

## 5. Next steps

### - As you might have noticed, we are not considering plurals here. Can you think of a way to account for this?
### - As highligthed throughout this notebook, our implementation of TF-IDF is not scalable, since we are collecting in main memory the IDF dictionary. 

### One way to alliviate this problem could be to split the IDF RDD into chunks (that we now for sure that will fit in memory), and join each of these chunks with the Term Frequency RDD in an iterative process. This is known as a *map-join*, since the join is performed in the mapping side, and it's use it typical when performing a join between a large RDD and a smaller one. 

### The implementation in Spark can be done with [broadcast variables](http://spark.apache.org/docs/latest/programming-guide.html#broadcast-variables). This is a more advanced feature of Spark, so this excercise could be a perfect topic for a Master's Thesis :) As an example, see the following  [explanation of a map-join in Spark](http://dmtolpeko.com/2015/02/20/map-side-join-in-spark/).

### - As we saw during the lesson, Spark's implementation of TF-IDF is not very robust (actually, it's still marked as *experimental*). The reason for this is that, in order to reduce the complexity of the problem, they use the so called [*hashing trick*](https://en.wikipedia.org/wiki/Feature_hashing). However, as explained in [Spark's documentation](http://spark.apache.org/docs/latest/mllib-feature-extraction.html#tf-idf), such implementation "suffers from potential hash collisions, where different raw features (i.e. words) may become the same term after hashing". This is precisely what we observed in the class.

### - Apart form querying on the corpus, one thing that could be done is to perform some clustering. For this, we could use the tf-idf index to calculate the distance between any pair of documents. Once this is done, we could build the [similarity matrix](https://en.wikipedia.org/wiki/Similarity_measure) and find clusters using, for instance, PCA, k-means, etc. 

### Another approach, more advanced and interesting, is to apply [topic model](https://en.wikipedia.org/wiki/Topic_model) to the corpus. The idea is to find the *latent* topics in the corpus. Every document might then belong to one or more topics (i.e., clusters). One famous algorithm for Topic Model is [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), and there are implementations of it in Python or R, as well as a [scalable version in Spark](http://spark.apache.org/docs/latest/mllib-clustering.html#latent-dirichlet-allocation-lda) (experimental, so be careful!).